### Урок 5. Задача оттока: варианты постановки, возможные способы решения

#### Задание

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)

2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)

3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.

4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3

5. (опционально) Еще раз провести оценку экономической эффективности

In [29]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

Ссылка на google drive: https://drive.google.com/file/d/1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91

In [30]:
#!wget 'https://drive.google.com/uc?export=download&id=1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91' -O churn_data.csv

In [31]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Есть как категориальные, так и вещественные признаки. Поле CustomerId нужно будет удалить. 

Посмотрим на распределение классов:

In [32]:
df.drop(columns = ['CustomerId'],axis = 1)

,RowNumber,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [33]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Не самое плохое распределение (1 к 4)

Давайте построим модель. Сразу же будем работать с использованием sklearn pipeline

In [34]:

X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

- Категориальные признаки закодируем с помощью OneHotEncoding
- Вещественные оставим пока как есть

In [35]:
#соберем  pipeline, понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    

class NumberSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
    
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [36]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


Зададим списки признаков

In [37]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion


Теперь нам нужно под каждый признак создать трансформер и объединить их в список.

In [39]:
from sklearn.preprocessing import StandardScaler

final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    
    final_transformers.append((cont_col, cont_transformer))


feats = FeatureUnion(final_transformers)

### 1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями:
    - бустинг
    - логистическая регрессия

##### 1. LogisticRegression()

In [40]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [41]:
model_lr = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state=42)),
])

# обучим пайплайн на всем тренировочном датасете
model_lr.fit(X_train, y_train)

preds = model_lr.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# находим индекс самого большого f-score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.28952195521688073, F-Score=0.510, Precision=0.462, Recall=0.568


In [42]:
metrics_df = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])
metrics_df

,model,thresh,F-Score,Precision,Recall,ROC AUC


In [43]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.7720774921330664

In [45]:
metrics_df = metrics_df.append({
    'model': type(model_lr['classifier']).__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\Инна\AppData\Local\Temp\ipykernel_14012\3349657576.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077


##### 2. GradientBoostingClassifier()

In [47]:
from sklearn.ensemble import GradientBoostingClassifier


model_gb = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(random_state=42)),
])


# обучим пайплайн на всем тренировочном датасете
model_gb.fit(X_train, y_train)

preds = model_gb.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# находим индекс самого большого f-score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.4085078904556646, F-Score=0.646, Precision=0.704, Recall=0.597


In [48]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.8757458662211781

In [49]:
metrics_df = metrics_df.append({
    'model': type(model_gb['classifier']).__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\Инна\AppData\Local\Temp\ipykernel_14012\900184340.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077
1,GradientBoostingClassifier,0.408508,0.646121,0.703704,0.59725,0.875746


### 2. Отобрать лучшую модель по метрикам (какая по вашему мнению здесь наиболее подходящая ML-метрика)


In [50]:
metrics_df.sort_values('F-Score')

,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077
1,GradientBoostingClassifier,0.408508,0.646121,0.703704,0.59725,0.875746


Цель – минимизация ложно-отрицательного решения, которое делает наше значение менее точным.

Упрощая: Precision – не прихватить лишнее. Recall – не пропустить нужное.

Очевидно, что чем выше Precision и Recall, тем лучше. Однако, в реальной жизни невозможно достичь максимума обоих показателей одновременно. Нам приходится искать баланс.

Таким образом, необходима метрика, которая объединяет в себе информацию о точности и полноте нашего алгоритма. Ей является F-score. По полученным данным становится понятнр, что модель GradientBoostingClassifier работает лучше Логистической регрессии, на ней мы и остановимся.

### 3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2:
    - 1 доллар на удержание
    - 2 доллара - с каждого правильно классифицированного (True Positive)

In [51]:
preds = model_gb.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds > thresholds[ix])

In [52]:
TN = cnf_matrix[0][0]
FP = cnf_matrix[0][1]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]


retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

175

### 4. *Провести подбор гиперпараметров лучшей модели по итогам 2-3

In [53]:
from sklearn.model_selection import GridSearchCV

params = {
    'classifier__max_features': [0.3, 0.5, 0.7],
    'classifier__min_samples_leaf': [1, 15, 30, 50],
    'classifier__n_estimators': [50, 100, 150, 300]
}

In [55]:
# проведем поиск гиперпараметров

grid = GridSearchCV(model_gb,
                    param_grid=params,
                    cv=5,
                    scoring='recall',
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__max_features': 0.7,
 'classifier__min_samples_leaf': 30,
 'classifier__n_estimators': 300}

In [67]:
from sklearn.ensemble import GradientBoostingClassifier

#подставим гиперпараметры в нашу модель
model_gb_tuned = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(n_estimators=300,
                                              min_samples_leaf=1,
                                              max_features=0.7,
                                              random_state=42)),
])


# обучим пайплайн на всем тренировочном датасете
model_gb_tuned.fit(X_train, y_train)

preds = model_gb_tuned.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# и снова находим индекс самого большого f-score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')                                                                        

Best Threshold=0.3642250281425497, F-Score=0.644, Precision=0.648, Recall=0.640


In [68]:
roc_auc = roc_auc_score(y_test, preds)
roc_auc

0.8727604278190957

In [69]:
metrics_df = metrics_df.append({
    'model': type(model_gb_tuned['classifier']).__name__,
    'thresh': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

C:\Users\Инна\AppData\Local\Temp\ipykernel_14012\271650954.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.289522,0.5097,0.4624,0.56778,0.772077
1,GradientBoostingClassifier,0.408508,0.646121,0.703704,0.59725,0.875746
2,LogisticRegression,0.347565,0.640777,0.633397,0.64833,0.873763
3,GradientBoostingClassifier,0.347565,0.640777,0.633397,0.64833,0.873763
4,GradientBoostingClassifier,0.347565,0.640777,0.633397,0.64833,0.873763
5,GradientBoostingClassifier,0.364225,0.644269,0.648111,0.640472,0.87276


Показатели Precision и Recall заметно выровнялись относительно др.др, ROC AUC и F-Score	 чуть ниже, чем до обработки, но практически такой же, что в сумме дает более сбалансированную модель.

5. *Еще раз провести оценку экономической эффективности

In [70]:
preds = model_gb_tuned.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

cnf_matrix = confusion_matrix(y_test, preds > thresholds[ix])

In [71]:
TN = cnf_matrix[0][0]
FP = cnf_matrix[0][1]
FN = cnf_matrix[1][0]
TP = cnf_matrix[1][1]


retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

148

Вывод: просматривая чистые метрики, можно прийти к выводу, что обработка GridSearchCV с поиском гиперпараметров
дает более сбалансированную модель. Однако, применив лучшие параметры по мнению Модели

{'classifier__max_features': 0.7,

'classifier__min_samples_leaf': 30,

'classifier__n_estimators': 300}

и посчитав экономическую эффективность, я получила меньший результат, чем в модели без просчитанных гиперпараметров. 
Выручка значительно снизилась. (было 175, стало 138).
Снова обучив модель с изменением одного гиперпараметра (число листьев: min_samples_leaf=1,) с рекомендованного на начальный (1),
я убедилась, что это значительно влияет на работу модели в лучшую сторону. (было 138, стало 148).